# Quantium - Task 2: Experimentation and uplift testing

## Dependencies

Required packages to run this notebook. 

In [1]:
!pip install pandas

## Loading datasets

In [2]:
import pandas as pd

pd.options.mode.chained_assignment = None

In [6]:
import chardet

with open('data/QVI_data.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# Check what the character encoding might be
result

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

In [7]:
df_data = pd.read_csv('data/QVI_data.csv', encoding='ascii')
df_data

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream
...,...,...,...,...,...,...,...,...,...,...,...,...
264829,2370701,2018-12-08,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,210,GRNWVES,YOUNG FAMILIES,Mainstream
264830,2370751,2018-10-01,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,KETTLE,YOUNG FAMILIES,Premium
264831,2370961,2018-10-24,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,165,TYRRELLS,OLDER FAMILIES,Budget
264832,2370961,2018-10-27,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,300,OLD,OLDER FAMILIES,Budget


## Control store selection

First we must select a control store to compare its performance against the stores in trial.

In [10]:
# How many stores are to choose from?
df_data.STORE_NBR.nunique()

272

Let's gather some metrics about the trials stores

In [29]:
# Average sales revenue per month per store
df_data['MONTH'] = pd.DatetimeIndex(df_data['DATE']).month
df_data[(df_data.STORE_NBR == 77) | (df_data.STORE_NBR == 86) | (df_data.STORE_NBR == 88)].groupby(['STORE_NBR','MONTH'])['TOT_SALES'].sum().groupby('STORE_NBR').mean()

STORE_NBR
77     253.333333
86     886.279167
88    1361.104167
Name: TOT_SALES, dtype: float64

In [31]:
# Total number of customers per store
df_data[(df_data.STORE_NBR == 77) | (df_data.STORE_NBR == 86) | (df_data.STORE_NBR == 88)].groupby(['STORE_NBR'])['LYLTY_CARD_NBR'].nunique()

STORE_NBR
77    356
86    273
88    388
Name: LYLTY_CARD_NBR, dtype: int64

In [33]:
# Average number of transactions per customer
df_data[(df_data.STORE_NBR == 77) | (df_data.STORE_NBR == 86) | (df_data.STORE_NBR == 88)].groupby(['STORE_NBR','LYLTY_CARD_NBR'])['LYLTY_CARD_NBR'].count().groupby('STORE_NBR').mean()

STORE_NBR
77    1.581461
86    5.633700
88    4.827320
Name: LYLTY_CARD_NBR, dtype: float64

With this in mind we should now look for similar or correlated stores

In [ ]:
corr = df.corr()

mask = np.triu(np.ones_like(corr, dtype=np.bool))

f, ax = plt.subplots(figsize=(20, 20))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, center=0,square=True, vmax=1.0,vmin=-1.0,linewidths=.5)